In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 读写文件

- 希望保存训练的模型，以备将来在各种环境中使用（比如在部署中进行预测）
- 当运行一个耗时较长的训练过程时，最佳的做法是定期保存中间结果

## 加载和保存张量

- 对单个张量，可以直接调用`load`和`save`函数分别读写它们

In [3]:
x = torch.arange(4)
torch.save(x, 'x-file')  # 存储的文件名称为x-file

- 将存储在文件中的数据读回内存

In [4]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

- 存储一个**张量列表**

In [5]:
y = torch.zeros(4)
torch.save([x, y],'x-files')

- 将张量列表读回内存

In [6]:
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

- 存储从字符串映射到张量的字典

In [7]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')

- 将张量字典读回到内存中

In [8]:
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 加载和保存模型参数

- 深度学习框架提供了内置函数来保存和加载整个网络

- 需要注意的是，这将保存模型的**参数**而**不是保存整个模型**

- 为了恢复模型，需要用代码生成模型架构，然后从磁盘加载参数

In [9]:
# 定义一个三层感知机

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()  # 实例化自定义的感知机
X = torch.randn(size=(2, 20))  # 输入数据
Y = net(X)

print(f'输出Y为\n{Y}')

输出Y为
tensor([[-0.0802,  0.2133, -0.1663,  0.1090, -0.1764, -0.0980, -0.0377,  0.0478,
          0.3004,  0.0946],
        [-0.1499,  0.2987,  0.1000, -0.0628, -0.1497, -0.2791, -0.1915, -0.0105,
          0.1062,  0.3925]], grad_fn=<AddmmBackward0>)


- 将模型参数存储在“mlp.params”的文件中

In [10]:
torch.save(net.state_dict(), 'mlp.params')

# 注意，net.state_dict()可以获得模型的所有参数

- 为了恢复模型，**实例化原始多层感知机模型**
- 不需要随机初始化模型参数，而是**直接读取文件中存储的参数**

In [11]:
clone = MLP()  # 建立模型的架构
clone.load_state_dict(torch.load('mlp.params'))  # 加载模型参数
clone.eval()   # 查看模型

<All keys matched successfully>

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

- ```python
torch.nn.Module.load_state_dict(state_dict,strict=True)
```
    - 将`state_dict`保存的模型参数注入到块中
    - `strict`为`True`，强制确保`state_dict`中的`keys`与模型`state_dict()`函数中保存的`keys`一致

In [12]:
# 检验加载的模型参数，给定同样的输入X，两个模型的输出应当一样

Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 加载和保存模型架构+参数

```python
torch.save(model, path)
```

- 保存模型+参数

In [13]:
torch.save(net,'mlp')

```python
torch.load(path)
```

- 加载模型

In [14]:
net2 = torch.load('mlp')

In [16]:
# 检验加载的模型，给定同样的输入X，net2和net的输出应当一样

Y_2 = net2(X)
Y_2 == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])